### Mnist classification with a basic CNN

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [4]:
# Same dataset as in the mnist_mlp.ipynb file
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root="./data", train=True, download=False, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Contrairement au MLP on a en entrée une image 2D de dimension carrée 28 (pour le MLP on avait un vecteur de taille 28*28)

In [8]:
# Create a simple CNN so it does not take took long to train
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        # les couches convolutionnelles servent à extraire les caractéristiques visuelles
        self.conv_layers = nn.Sequential(
            
            # 1ère couche: 1 canal en entrée (image en niveaux de gris)
            # 32 filtres/features en sortie de couche ; Les filtres sont de taille 3 par 3
            nn.Conv2d(1, 32, kernel_size=3),
            
            # Sans activation, empiler les couches convolutives revient à une seule grande convolution linéaire
            nn.ReLU(),
            
            # La couche combine les 32 features pour détecter des motifs plus complexes et plus abstraits 
            # (combinaison de bords, boucle etc)
            nn.Conv2d(32, 64, kernel_size=3),
            
            nn.ReLU(),
            
            # Couche de pooling pour réduire la dimension (cf max-pooling, avg-pooling je vois le principe)
            nn.MaxPool2d(2)
        )
        
        # les couches fully connected utilisent les caractéristiques pour classifier
        self.fc_layers = nn.Sequential(
            # Transforme les features 3D en vecteur 1D pour les couches linéaires
            nn.Flatten(),
            
            # Il y a 64*12*12=? features extraites par les convolutions
            # On met 128 neurones en sortie
            # La couche append à combiner les features visuelles pour prendre des décisions
            nn.Linear(64*12*12, 128),
            
            nn.ReLU(),
            
            # 10 neurones pour la couche de sortie
            nn.Linear(128, 10)
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x
        
model = CNN()

In [ ]:
# Optimizer and loss functioooonnnnn ; always the same I get it !
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training
for epoch in range(3):
    for images, labels in train_loader:
        logits = model(images)
        loss = criterion(logits, labels)
        
        # always the same thing, Pytorch automatically add the gradients to their old value, so I've to set them to 0 !
        optimizer.zero_grad() 
        
        loss.backward()
        optimizer.step()
        
    print(f"Epoch {epoch}, Loss: {loss.item():.4f}")        

Epoch 0, Loss: 0.0399
Epoch 1, Loss: 0.0339
Epoch 2, Loss: 0.0586


In [11]:
# Testing the function
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        preds = model(images).argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Accuracy = {100 * correct / total:.2f}%")

Accuracy = 98.83%


Better results than with simple MLP, however the training time is much longer